<a href="https://colab.research.google.com/github/camilafaiza26/kelompok8_apm/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=79300591fbb6f63cc89d2ff7ba49f83c87b104b9da1e980bf35e21d3a4f5331a
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [4]:
# import library
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [5]:
# import dataset
df = pd.read_excel("data_retail2.xlsx")
df.head()

,InvoiceNo,InvoiceDate,BRANCH_SPLR,BRANCHNAME_SPLR,warehouseProductsID,BARCODEID,StockCode,PRODUCT,PRODUCT_CATEGORY,Quantity,...,CustomerID,CUSTNAME,ADDRESS,KOTA,PROVINSI,NEGARA,CHANNELID_SPLR,CHANNELNAME_SPLR,SUBDISTID,SUBDIST_NAME
0,536365,2020-12-01 08:26:00,13,SAMARINDA,10001,8992753282401,85123A,123 BENDERA COKLAT 300G,SUSU,6,...,17850.0,DUA PUTRI SLAMET RIYADI,JL. SLAMET RIYADI,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
1,536365,2020-12-01 08:26:00,13,SAMARINDA,20020,8999909192034,71053,2.3.4 FILTER,ROKOK,6,...,17850.0,SANURI,JL. M. SAID,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
2,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20021,8999909007147,84406B,234 KERETEK,ROKOK,8,...,17850.0,EMI MBAK,PS. TALUN NO.63,MAGELANG,JAWA TENGAH,INDONESIA,32,Toko Kelontong,190105,PT. KTRI DISTRIBUSI
3,536365,2020-12-01 08:26:00,13,SAMARINDA,20018,8999909172234,84029G,234 KERETEK 12,ROKOK,6,...,17850.0,RISKA CELL ADAM MALIK,JL. ADAM MALIK,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
4,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20019,8999909005860,84029E,234 KERETEK 16,ROKOK,6,...,17850.0,ABADI MART,JL.GROWONG - PUCUNG REJO MUNTILAN.,MAGELANG,JAWA TENGAH,INDONESIA,42,Mini Market,190105,PT. KTRI DISTRIBUSI


In [6]:
# mengganti kolom PERIODE menjadi tipe data datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [7]:
# membersihkan ruang di deskripsi produk, kategori produk, 
# dan menghapus baris yang tidak memiliki invoice yang valid
df["PRODUCT"] = df["PRODUCT"].str.strip()
df["PRODUCT_CATEGORY"] = df["PRODUCT_CATEGORY"].str.strip()

df.dropna(axis=0, subset=["InvoiceNo"], inplace=True)

In [8]:
# menghapus variabel invoiceNo yang diawali dengan huruf C pada invoice numbernya
df["InvoiceNo"] = df["InvoiceNo"].astype("str")
df = df[~(df["InvoiceNo"].str[0] == "C")]

In [9]:
basket = (df[df['PROVINSI'] == "JAWA TENGAH"].groupby(['InvoiceNo', 'PRODUCT_CATEGORY'])['Quantity'].count()\
                                    .unstack().reset_index().fillna(0)\
                                    .set_index('InvoiceNo'))
basket.head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
# menampilkan subset kolom
basket.iloc[:,[0,1,2,3,4,5,6,7]].head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN
InvoiceNo,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
# melakukan proses encoding -> mengubah data ke bentuk angka, agar sistem atau komputer dapat memahami informasi dari dataset
def encode_units(x):
  if x <= 0:
    return 0

  if x >= 1:
    return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head(5)

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
# membangun frequent items, rules, dan model
frequent_itemsets = apriori(basket_sets, min_support = 0.1, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.326438,(ALAT RUMAH TANGGA)
1,0.348726,(BISKUIT)
2,0.172516,(BUMBU)
3,0.166359,(COKELAT)
4,0.273488,(DETERGEN)
...,...,...
250,0.102204,"(MINUMAN, SNACK, KOSMETIK, SABUN & SAMPHOO)"
251,0.117473,"(MINUMAN, KOSMETIK, SUSU, SABUN & SAMPHOO)"
252,0.102081,"(PARFUM, OBATAN, KOSMETIK, SABUN & SAMPHOO)"
253,0.106760,"(PARFUM, KOSMETIK, SUSU, SABUN & SAMPHOO)"


In [13]:
rules1 = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BISKUIT),(ALAT RUMAH TANGGA),0.348726,0.326438,0.177811,0.509887,1.561974,0.063973,1.374300
1,(ALAT RUMAH TANGGA),(BISKUIT),0.326438,0.348726,0.177811,0.544700,1.561974,0.063973,1.430430
2,(DETERGEN),(ALAT RUMAH TANGGA),0.273488,0.326438,0.140377,0.513282,1.572375,0.051100,1.383887
3,(ALAT RUMAH TANGGA),(DETERGEN),0.326438,0.273488,0.140377,0.430026,1.572375,0.051100,1.274641
4,(KOSMETIK),(ALAT RUMAH TANGGA),0.412757,0.326438,0.181997,0.440931,1.350735,0.047258,1.204792


In [14]:
# melakukan filter untuk nilai lift ratio minimal adalah 1 dan tingkat confidence minimal 0.8
result1 = rules1[(rules1['lift'] >= 1) &
                 (rules1['confidence'] >= 0.8)] 

apr_result = result1.sort_values(by = 'confidence', ascending = False)
apr_result.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1239,"(PARFUM, OBATAN, SABUN & SAMPHOO)",(KOSMETIK),0.115503,0.412757,0.102081,0.883795,2.141200,0.054406,5.053522
1254,"(PARFUM, SUSU, SABUN & SAMPHOO)",(KOSMETIK),0.121290,0.412757,0.106760,0.880203,2.132497,0.056697,4.901986
1086,"(BISKUIT, OBATAN, SABUN & SAMPHOO)",(MINUMAN),0.117596,0.383327,0.102697,0.873298,2.278206,0.057619,4.867128
1183,"(PARFUM, MINUMAN, SABUN & SAMPHOO)",(KOSMETIK),0.140500,0.412757,0.122522,0.872042,2.112725,0.064530,4.589344
1057,"(BISKUIT, PARFUM, SABUN & SAMPHOO)",(KOSMETIK),0.127694,0.412757,0.111316,0.871745,2.112006,0.058610,4.578729
1169,"(KOSMETIK, OBATAN, SUSU)",(MINUMAN),0.120552,0.383327,0.104913,0.870276,2.270321,0.058702,4.753722
1043,"(BISKUIT, OBATAN, SABUN & SAMPHOO)",(KOSMETIK),0.117596,0.412757,0.102327,0.870157,2.108158,0.053789,4.522718
1072,"(BISKUIT, SUSU, SABUN & SAMPHOO)",(KOSMETIK),0.119074,0.412757,0.103559,0.869700,2.107051,0.054410,4.506857
1211,"(MINUMAN, SNACK, SABUN & SAMPHOO)",(KOSMETIK),0.118212,0.412757,0.102204,0.864583,2.094654,0.053411,4.336563
1155,"(MINUMAN, OBATAN, SABUN & SAMPHOO)",(KOSMETIK),0.133481,0.412757,0.115257,0.863469,2.091954,0.060161,4.301158
